In [346]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices

### SETS

In [347]:

# Velg antall av de ulike settene her
set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 5)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 5)
f_values = range(1, 8)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 2294)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, 19) # 18 installations
s_values = range(1,12) # 11 systems
w_values = range(1, 4) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6
7


In [348]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = 'SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [349]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)

#### Inputdata

In [350]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.29, 2:0.30, 3:0.82, 4:0.73, 5:0.87, 6:0.69, 7: 0.5}

fuel_dict = {
    'MGO': (0.18, 0.09, 'fossil'),
    'LNG': (0.15, 0.07, 'fossil'),
    'B100': (0.51, 0.27, 'bio'),
    'FB30': (0.28, 0.15, 'bio'),
    'methanol': (0.47, 0.26, 'low_carbon'),
    'NH3': (0.41, 0.22, 'low_carbon'),
    'El': (0.78, 0.40, 'E'),
}  # Example fuel bounds with group information

# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000181667, 2: 0.0001525, 3: 0.000109444, 4: 0.00004775, 5: 0.0000722222, 6: 0.00000791667}
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0}
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw 


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {0: 0, 1: 0, 2: 0.50, 3: 0.55, 4: 0.70, 5: 0.80, 6: 0.95} # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {0: 0, 1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

initial_fleet = 1

#### Lese inn data fra rutegenereringsprogrammet

In [351]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation/generated_datafiles/mj_route.csv')
mj_route_el = pd.read_csv('../route_generation/generated_datafiles/mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv('../route_generation/generated_datafiles/hours_route.csv')
hours_route_el = pd.read_csv('../route_generation/generated_datafiles/hours_route_strøm.csv')



# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = 'Parameterdata-sheets.xlsx'

#### Initial fleet

In [352]:
# Create a DataFrame from the combinations
# df_initialfleet = pd.DataFrame(combinations_s_a, columns=['S', 'A'])
#df_initialfleet['Value'] = [0]


# Set chosen values in 'Value' column
# df_initialfleet.loc[(df_initialfleet['S'] == 1) & (df_initialfleet['A'] == 3), 'Value'] = 2
# df_initialfleet.loc[(df_initialfleet['S'] == 2) & (df_initialfleet['A'] == 4), 'Value'] = 1


# Using ExcelWriter to write to the existing Excel file
# with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     df_initialfleet.to_excel(writer, sheet_name='Initial fleet', index=False)

# df_initialfleet

##### Cost of fuel per route

In [353]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route.to_dict()
distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

specific_distance_dict = mj_route_el.to_dict()
specific_distance_dict= specific_distance_dict['mj_route']
electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(distance_dict)
mj_route

print(electric_distance_dict)

{0: 226582.0, 1: 246422.0, 2: 199467.2, 3: 215307.2, 4: 184089.6, 5: 211865.2, 6: 221785.2, 7: 215145.2, 8: 228025.2, 9: 187132.4, 10: 225292.4, 11: 194265.2, 12: 204502.0, 13: 197372.4, 14: 218742.0, 15: 198022.4, 16: 192432.0, 17: 195125.2, 18: 194662.0, 19: 209932.4, 20: 242172.4, 21: 215865.2, 22: 225382.0, 23: 219132.4, 24: 238902.0, 25: 220742.4, 26: 215392.0, 27: 217845.2, 28: 216742.0, 29: 227614.8, 30: 243600.0, 31: 248400.0, 32: 222000.0, 33: 238565.2, 34: 269612.8, 35: 265292.8, 36: 237760.0, 37: 277360.0, 38: 249292.8, 39: 245129.6, 40: 253789.6, 41: 234640.0, 42: 259289.6, 43: 223530.0, 44: 214019.6, 45: 232169.6, 46: 249689.6, 47: 252329.6, 48: 272094.8, 49: 248724.8, 50: 261849.6, 51: 280974.8, 52: 253844.8, 53: 229792.4, 54: 267952.4, 55: 240032.4, 56: 237785.2, 57: 253552.4, 58: 254432.4, 59: 230894.8, 60: 211924.8, 61: 183360.0, 62: 180240.0, 63: 178492.0, 64: 180170.0, 65: 180640.0, 66: 254432.4, 67: 252505.2, 68: 264189.6, 69: 259869.6, 70: 232325.2, 71: 271925.2, 7

Generere alle kombinasjoner av R og F

In [354]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [355]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
def calculate_value(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

df_fuelprice

,F,R,Value
0,1,1,65708.780
1,1,2,71462.380
2,1,3,57845.488
3,1,4,62439.088
4,1,5,53385.984
...,...,...,...
16046,7,2289,116213.800
16047,7,2290,125253.800
16048,7,2291,114893.800
16049,7,2292,132453.600


##### Fuelcost 2

In [356]:
# def generate_fuel_prices(scenario_range, period_range, fuel_dict):
#     num_scenarios = len(scenario_range)
#     num_periods = len(period_range)
#     num_fuels = len(fuel_dict)
    
#     # Generate random scaling factors for each group
#     group_scaling_factors = {}
#     for fuel_key, (_, _, group) in fuel_dict.items():
#         if group not in group_scaling_factors:
#             group_scaling_factors[group] = np.random.uniform(0, 1, (num_scenarios, num_periods))
    
#     # Generate random prices for each fuel within bounds for each time period in each scenario
#     fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))
#     for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
#         for s_idx, scenario in enumerate(scenario_range):
#             for p_idx, period in enumerate(period_range):
#                 scaling_factor = group_scaling_factors[group][s_idx, p_idx]
#                 fuel_prices[i, p_idx, s_idx] = lower_bound + (upper_bound - lower_bound) * scaling_factor
#                 if period > 0:
#                     fuel_prices[i, p_idx, s_idx] = max(fuel_prices[i, p_idx, s_idx], fuel_prices[i, p_idx-1, s_idx])
    
#     # Grouping fuels into four groups
#     fuel_groups = {}
#     for fuel_key, (_, _, group) in fuel_dict.items():
#         if group not in fuel_groups:
#             fuel_groups[group] = []
#         fuel_groups[group].append(fuel_key)
    
#     # Generating correlated prices within each group
#     for group, fuels_in_group in fuel_groups.items():
#         # Generating correlated random numbers for this group
#         num_group_fuels = len(fuels_in_group)
#         means = np.random.uniform(0, 1, num_group_fuels)
        
#         # Define covariance matrix with specified correlation
#         if group == 'Group A':
#             # For Group A, set 100% correlation
#             cov_matrix = np.eye(num_group_fuels)
#         else:
#             # For other groups, set 70% correlation
#             cov_matrix = np.full((num_group_fuels, num_group_fuels), 0.7)
#             np.fill_diagonal(cov_matrix, 1.0)
        
#         # Generate correlated random numbers using Cholesky decomposition
#         correlated_values = np.random.multivariate_normal(means, cov_matrix, num_scenarios)
        
#         # Normalizing the correlated values
#         correlated_values -= correlated_values.min(axis=0)
#         correlated_values /= correlated_values.max(axis=0)
        
#         # Scaling the correlated values to the group bounds
#         for i, fuel_key in enumerate(fuels_in_group):
#             lower_bound, upper_bound, _ = fuel_dict[fuel_key]
#             for s_idx, scenario in enumerate(scenario_range):
#                 fuel_prices[list(fuel_dict.keys()).index(fuel_key), :, s_idx] = lower_bound + (upper_bound - lower_bound) * correlated_values[s_idx, i]
    
#     return fuel_prices

In [357]:
def generate_fuel_prices(scenario_range, period_range, fuel_dict):
    num_scenarios = len(scenario_range)
    num_periods = len(period_range)
    num_fuels = len(fuel_dict)
    
    # Generate random scaling factors for each group
    group_scaling_factors = {}
    for fuel_key, (_, _, group) in fuel_dict.items():
        if group not in group_scaling_factors:
            group_scaling_factors[group] = np.random.uniform(0, 1, (num_scenarios, num_periods))
    
    # Generate random prices for each fuel within bounds for each time period in each scenario
    fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))
    for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
        for s_idx, scenario in enumerate(scenario_range):
            for p_idx, period in enumerate(period_range):
                scaling_factor = group_scaling_factors[group][s_idx, p_idx]
                fuel_price = lower_bound + (upper_bound - lower_bound) * scaling_factor
                if p_idx > 0:
                    prev_fuel_price = fuel_prices[i, p_idx - 1, s_idx]
                    fuel_price = max(fuel_price, prev_fuel_price)
                fuel_prices[i, p_idx, s_idx] = fuel_price
    
    # Grouping fuels into four groups
    fuel_groups = {}
    for fuel_key, (_, _, group) in fuel_dict.items():
        if group not in fuel_groups:
            fuel_groups[group] = []
        fuel_groups[group].append(fuel_key)
    
    # Generating correlated prices within each group
    for group, fuels_in_group in fuel_groups.items():
        # Generating correlated random numbers for this group
        num_group_fuels = len(fuels_in_group)
        means = np.random.uniform(0, 1, num_group_fuels)
        
        # Define covariance matrix with specified correlation
        if group == 'El':
            # For Group A, set 100% correlation
            cov_matrix = np.eye(num_group_fuels)
        else:
            # For other groups, set 70% correlation
            cov_matrix = np.full((num_group_fuels, num_group_fuels), 0.7)
            np.fill_diagonal(cov_matrix, 1.0)
        
        # Generate correlated random numbers using Cholesky decomposition
        correlated_values = np.random.multivariate_normal(means, cov_matrix, num_scenarios)
        
        # Normalizing the correlated values
        correlated_values -= correlated_values.min(axis=0)
        correlated_values /= correlated_values.max(axis=0)
        
        # Scaling the correlated values to the group bounds
        for i, fuel_key in enumerate(fuels_in_group):
            lower_bound, upper_bound, _ = fuel_dict[fuel_key]
            for s_idx, scenario in enumerate(scenario_range):
                fuel_prices[list(fuel_dict.keys()).index(fuel_key), :, s_idx] = lower_bound + (upper_bound - lower_bound) * correlated_values[s_idx, i]
    
    return fuel_prices



In [358]:

# Generate fuel prices

fuel_prices = generate_fuel_prices(w_values, t2_values, fuel_dict)

print(fuel_prices)
# Get the first value from each range
first_scenario = next(iter(w_values))
first_period = next(iter(t2_values))


# Create DataFrame to store parameter values
data = []
for fuel_type, (_, _, group) in fuel_dict.items():
    distances = specific_distance_dict if group == 'El' else distance_dict
    for route, distance in distances.items():
        # Ensure the period index is within bounds
        period_idx = min(first_period, len(t2_values))
        scenario_idx = min(first_scenario, len(w_values))
        fuel_price = fuel_prices[scenario_idx - 1, period_idx - 1, list(fuel_dict.keys()).index(fuel_type)]
        data.append([fuel_type, route, period_idx, scenario_idx, fuel_price * distance])

parameters_df = pd.DataFrame(data, columns=['fuel type', 'route', 'time period', 'scenario', 'Value'])
print(parameters_df)


[[[0.09       0.13389235 0.18      ]
  [0.09       0.13389235 0.18      ]
  [0.09       0.13389235 0.18      ]]

 [[0.15       0.07       0.10886032]
  [0.15       0.07       0.10886032]
  [0.15       0.07       0.10886032]]

 [[0.51       0.27       0.39946295]
  [0.51       0.27       0.39946295]
  [0.51       0.27       0.39946295]]

 [[0.28       0.15       0.21023775]
  [0.28       0.15       0.21023775]
  [0.28       0.15       0.21023775]]

 [[0.44267869 0.26       0.47      ]
  [0.44267869 0.26       0.47      ]
  [0.44267869 0.26       0.47      ]]

 [[0.41       0.22       0.23541462]
  [0.41       0.22       0.23541462]
  [0.41       0.22       0.23541462]]

 [[0.4        0.58249804 0.78      ]
  [0.4        0.58249804 0.78      ]
  [0.4        0.58249804 0.78      ]]]


IndexError: index 3 is out of bounds for axis 2 with size 3

In [ ]:
df_fuelprice2 = pd.DataFrame(combinations_f_r_w, columns=['F', 'R', 'O'])

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
def calculate_value(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

df_fuelprice2['Value'] = df_fuelprice2.apply(calculate_value, axis=1)

def adjust_value(row):
    if row['O'] == 1:
        return 0.75 * row['Value']  # 75% of the original value for O index 1
    elif row['O'] == 3:
        return 1.25 * row['Value']  # 125% of the original value for O index 3
    else:
        return row['Value']  # Unchanged for O index 2

# Apply the adjust_value function to create a new column 'Adjusted_Value'
df_fuelprice2['Value'] = df_fuelprice2.apply(adjust_value, axis=1)



df_fuelprice2

# Display the first few rows of the DataFrame
print(df_fuelprice2.head())

   F  R  O      Value
0  1  1  1  49281.585
1  1  1  2  65708.780
2  1  1  3  82135.975
3  1  2  1  53596.785
4  1  2  2  71462.380


##### Skriver Cost of fuel til excel

In [ ]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice2.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [ ]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [ ]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance
print(electric_distance_dict)
def calculate_emissions(row):
    distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['F']] * distance

print(df_ems)
# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)

df_ems

{1: 297142.0, 2: 325542.0, 3: 257947.2, 4: 280587.2, 5: 237129.6, 6: 275385.2, 7: 289545.2, 8: 280105.2, 9: 298505.2, 10: 241132.4, 11: 295612.4, 12: 250265.2, 13: 265622.0, 14: 255692.4, 15: 285942.0, 16: 256022.4, 17: 247792.0, 18: 252725.2, 19: 251542.0, 20: 273692.4, 21: 319692.4, 22: 281145.2, 23: 295462.0, 24: 286812.4, 25: 314742.0, 26: 288502.4, 27: 280592.0, 28: 285205.2, 29: 283142.0, 30: 298894.8, 31: 322480.0, 32: 329280.0, 33: 291600.0, 34: 314805.2, 35: 382572.8, 36: 376412.8, 37: 314080.0, 38: 394720.0, 39: 329532.8, 40: 324329.6, 41: 335469.6, 42: 309680.0, 43: 344569.6, 44: 293130.0, 45: 279299.6, 46: 305849.6, 47: 330889.6, 48: 334649.6, 49: 386414.8, 50: 329604.8, 51: 348249.6, 52: 399054.8, 53: 336964.8, 54: 300832.4, 55: 379312.4, 56: 315392.4, 57: 312425.2, 58: 334752.4, 59: 336032.4, 60: 303534.8, 61: 277044.8, 62: 236400.0, 63: 231920.0, 64: 229132.0, 65: 231210.0, 66: 232480.0, 67: 336032.4, 68: 333465.2, 69: 374349.6, 70: 368189.6, 71: 305845.2, 72: 386405.2, 

,F,R,Value
0,1,1,41.162472
1,1,2,44.766745
2,1,3,36.236608
3,1,4,39.114213
4,1,5,33.443005
...,...,...,...
13753,6,2289,2.411953
13754,6,2290,2.616520
13755,6,2291,2.382186
13756,6,2292,2.972450


##### MAX EMISSIONS PER TIME PERIOD

In [ ]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
print(df_max_ems)

   T
0  0
1  1
2  2
3  3
4  4


In [ ]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)
print(df_max_ems.head())
# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

   T     Value
0  0  165000.0
1  1  165000.0
2  2   82500.0
3  3   74250.0
4  4   49500.0


,T,Value
0,0,165000.0
1,1,165000.0
2,2,82500.0
3,3,74250.0
4,4,49500.0


##### TIME USED

In [ ]:
# LAGE TIME USED

# Create T_rs
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route.to_dict()
route_dict = route_dict['Hours']
hours_route_dict = {key + 1: value for key, value in route_dict.items()}

# lager egne distanser for elektrisk
hours_route_el_dict = hours_route_el.to_dict()
hours_route_el_dict= hours_route_el_dict['Hours']
hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}

print(hours_route_dict)
print(hours_route_el_dict)


# funksjon for å fylle inn verdier i Trs
def calculate_timeused(row):
    value = hours_route_el_dict[row['R']] if row['S'] == 11 else hours_route_dict[row['R']]
    return value

# Legger til verdiene i Value-kolonne i dataframen
df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 48.0, 2: 48.0, 3: 48.0, 4: 48.0, 5: 48.0, 6: 48.0, 7: 48.0, 8: 48.0, 9: 48.0, 10: 48.0, 11: 48.0, 12: 48.0, 13: 48.0, 14: 48.0, 15: 48.0, 16: 48.0, 17: 48.0, 18: 48.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 48.0, 25: 48.0, 26: 48.0, 27: 48.0, 28: 48.0, 29: 48.0, 30: 48.0, 31: 48.0, 32: 48.0, 33: 48.0, 34: 48.0, 35: 48.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 48.0, 41: 48.0, 42: 48.0, 43: 48.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 48.0, 50: 48.0, 51: 48.0, 52: 48.0, 53: 48.0, 54: 48.0, 55: 48.0, 56: 48.0, 57: 48.0, 58: 48.0, 59: 48.0, 60: 48.0, 61: 48.0, 62: 48.0, 63: 48.0, 64: 48.0, 65: 48.0, 66: 48.0, 67: 48.0, 68: 48.0, 69: 48.0, 70: 48.0, 71: 48.0, 72: 48.0, 73: 48.0, 74: 48.0, 75: 48.0, 76: 48.0, 77: 48.0, 78: 48.0, 79: 48.0, 80: 48.0, 81: 48.0, 82: 48.0, 83: 48.0, 84: 48.0, 85: 48.0, 86: 48.0, 87: 48.0, 88: 48.0, 89: 48.0, 90: 48.0, 91: 48.0, 92: 48.0, 93: 48.0, 94: 48.0, 95: 48.0, 96: 48.0, 97: 48.0, 98: 48.0, 99: 48.0, 100: 48.0, 101: 48